<a href="https://colab.research.google.com/github/apoorva-aa/IR_Project_2023/blob/main/Lexical_text_simplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentence-transformers
!pip install wordfreq
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=254ab2705c4f83c81d1d0d765961ba5dc42b8d69b64f704fc53f4f93a765930c
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.o

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import torch
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForMaskedLM
from wordfreq import zipf_frequency
from sklearn.metrics.pairwise import cosine_similarity
import textstat
import gensim.downloader as api
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
bert_similarity_model = SentenceTransformer('bert-base-nli-mean-tokens')
synonym_model = api.load("glove-wiki-gigaword-50")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [8]:
synonym_model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [42]:
import string
import itertools
from nltk.corpus import wordnet
import gensim.downloader as api

def sentence_complexity(sentence):
    wpt = WordPunctTokenizer()
    tokens = wpt.tokenize(sentence)
    complexity_sum = 0
    num_tokens = len(tokens)
    for i in range(num_tokens):
      complexity_sum+= zipf_frequency(tokens[i], 'en')
    return complexity_sum
def user_interface_function(text):
  def get_top_k_predictions(input_string, k,tokenizer,model) -> str:
    tokenized_inputs = tokenizer(input_string, return_tensors="tf")
    outputs = model(tokenized_inputs["input_ids"])
    top_k_indices = tf.math.top_k(outputs.logits, 7).indices[0].numpy()
    decoded_output = tokenizer.batch_decode(top_k_indices)
    mask_token = tokenizer.encode(tokenizer.mask_token)[1:-1]
    mask_index = np.where(tokenized_inputs['input_ids'].numpy()[0]==mask_token)[0][0]

    decoded_output_words = decoded_output[mask_index]
    return decoded_output_words    
  def get_synonym(word):
    if(word not in synonym_model.key_to_index):
      return word
    max_zip_frequency = 0
    synonym_index = 0
    synonym_list = synonym_model.most_similar(word, topn=10)
    synonyms = [synonym[0] for synonym in synonym_list]
    synonyms_similarity_measure = [synonym[1] for synonym in synonym_list]
    remove_words_list = []
    for i in range(len(synonyms)):
      if(zipf_frequency(synonyms[i], 'en') < zipf_frequency(word, 'en')):
        remove_words_list.append(i)
    remove_words_list.sort(reverse=True)
    for i in range(len(remove_words_list)):
          synonyms.pop(remove_words_list[i])
          synonyms_similarity_measure.pop(remove_words_list[i])
    if(len(synonyms) == 0):
      return word

    print("synonyms")
    print(synonyms)
    zipf_frequency_arr = []
    for i in range(len(synonyms)):
      zipf_frequency_arr.append(zipf_frequency(synonyms[i], 'en'))

    indexed_synonyms_similarity_measure = list(enumerate(synonyms_similarity_measure))
    sorted_sim_arr = sorted(indexed_synonyms_similarity_measure, key=lambda x: x[1],reverse = True)
    indexed_zipf_frequency = list(enumerate(zipf_frequency_arr))
    sorted_complex_arr = sorted(indexed_zipf_frequency, key=lambda x: x[1],reverse = True)
    mydict_sim = {}
    mydict_complex = {}
    for i in range(len(sorted_complex_arr)):
      mydict_sim[sorted_sim_arr[i][0]] = i + 1
      mydict_complex[sorted_complex_arr[i][0]] = i + 1
    min_rank = 100000000000000000
    required_index = 0
    print(mydict_sim)
    print(mydict_complex)
    for i in range(len(sorted_sim_arr)):
      rank_val = mydict_sim[i] * mydict_complex[i]
      print("{} {}".format(rank_val,synonyms[i]))
      if(min_rank > rank_val):
        min_rank = rank_val
        required_index = i
      if(min_rank == rank_val and mydict_complex[i] < mydict_sim[i]):
        required_index = i
    return synonyms[required_index]
  text = text.lower()
  wpt = WordPunctTokenizer()
  tokens = wpt.tokenize(text)
  pos_tags = nltk.pos_tag(tokens)
  complex_words = []
  index_of_complex_word = []
  for i in range(len(tokens)):
    if(textstat.difficult_words(tokens[i]) == 1):
      complex_words.append(tokens[i])
      index_of_complex_word.append(i)
  sentence1_transformed = bert_similarity_model.encode(text)
  best_2_word_list = []
  if(len(tokens) < 6):
    simpler_words = []
    new_sentence = text
    for j in range(len(complex_words)):
       new_sentence = new_sentence.replace(complex_words[j],get_synonym(complex_words[j]))
    return new_sentence
  else:
    for j in range(len(complex_words)):
      if(index_of_complex_word[j] != (len(tokens) -1) and pos_tags[index_of_complex_word[j]][1] == "JJ" and pos_tags[index_of_complex_word[j] + 1][1] == "NN"):
        best_2_word_list.append([get_synonym(complex_words[j])])
      else:
        text_with_mask = text.replace(complex_words[j], '[MASK]')
        predictedWords = get_top_k_predictions(text_with_mask,10,tokenizer,model)
        predictedWords = predictedWords.split()
        word = ''
        remove_words_list = []
        for i in range(len(predictedWords)):
          if(zipf_frequency(predictedWords[i], 'en') < zipf_frequency(complex_words[j], 'en')):
            remove_words_list.append(i)
          else:
            for letter in predictedWords[i]:
              if(letter in string.punctuation):
                remove_words_list.append(i)
                break
        remove_words_list.sort(reverse=True)
        for i in range(len(remove_words_list)):
          predictedWords.pop(remove_words_list[i])
        similarity = []
        for i in range(len(predictedWords)):
          if(complex_words[j] == predictedWords[i]):
            similarity.append(0)
          else:
            simpler_text = text.replace(complex_words[j], predictedWords[i])
            sentence2_transformed = bert_similarity_model.encode(simpler_text)
            similarity_val = cosine_similarity([sentence1_transformed],[sentence2_transformed])[0][0]
            similarity.append(similarity_val)
        if(len(similarity) == 0):
          best_2_word_list.append([complex_words[j]])
        else:
          best_word = predictedWords[similarity.index(max(similarity))]
          predictedWords.pop(similarity.index(max(similarity)))
          similarity.pop(similarity.index(max(similarity)))
          if(len(predictedWords) != 0):
            best_word_2 = predictedWords[similarity.index(max(similarity))]
            best_2_word_list.append([best_word,best_word_2])
          else:
            best_2_word_list.append([complex_words[j]])
    similarity_arr = []
    possible_combinations = list(itertools.product(*best_2_word_list))
    possible_combinations_complexity = []
    simpler_text_arr = []
    for i in range(len(possible_combinations)):
      simpler_text = text
      words_complexity = 0
      for j in range(len(complex_words)):
        simpler_text = simpler_text.replace(complex_words[j], possible_combinations[i][j])
        words_complexity += zipf_frequency(possible_combinations[i][j], 'en')
      simpler_text_arr.append(simpler_text)
      possible_combinations_complexity.append(words_complexity)
      sentence2_transformed = bert_similarity_model.encode(simpler_text)
      similarity_val = cosine_similarity([sentence1_transformed],[sentence2_transformed])[0][0]
      similarity_arr.append(similarity_val)
    indexed_similarity_arr = list(enumerate(similarity_arr))
    sorted_sim_arr = sorted(indexed_similarity_arr, key=lambda x: x[1],reverse = True)
    indexed_possible_combinations_complexity = list(enumerate(possible_combinations_complexity))
    sorted_complex_arr = sorted(indexed_possible_combinations_complexity, key=lambda x: x[1],reverse = True)
    mydict_sim = {}
    mydict_complex = {}
    for i in range(len(sorted_sim_arr)):
      mydict_sim[sorted_sim_arr[i][0]] = i + 1
      mydict_complex[sorted_complex_arr[i][0]] = i + 1
    min_rank = 100000000000000000
    required_index = 0
    for i in range(len(sorted_sim_arr)):
      rank_val = mydict_sim[i] * mydict_complex[i]
      if(min_rank > rank_val):
        min_rank = rank_val
        required_index = i
      if(min_rank == rank_val and mydict_complex[i] < mydict_sim[i]):
        required_index = i
    new_sentence = text
    for j in range(len(complex_words)):
        new_sentence = new_sentence.replace(complex_words[j], possible_combinations[required_index][j])
    return new_sentence

In [ ]:
input_text = "The honourable man was accused of robbery."
simplified_text = user_interface_function(input_text)

In [ ]:
print(simplified_text)
print(sentence_complexity(input_text))
print(sentence_complexity(simplified_text))